# Goodreads web scraping

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import json

import os
import sys

sys.path.append(os.path.abspath('../'))
import lib.goodreads_functions as gr


In [50]:
import importlib
importlib.reload(gr)

<module 'lib.goodreads_functions' from 'c:\\Users\\54189\\ironhack_262\\final project\\LitMental\\lib\\goodreads_functions.py'>

## get book urls

get the url and number of voting for each book

In [2]:
def get_award_urls(url):
    urls=[]
    try:
        open_url = urllib.request.urlopen(url)
        soup = bs(open_url, 'html.parser')
        award_elements = soup.find_all('div', class_='category clearFix')

        # Extract URLs from award elements
        for element in award_elements:
            link = element.find('a', href=True)
            if link:
                urls.append('https://goodreads.com' + link['href'])

    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return urls
    

In [4]:
def get_book_urls(url):
    urls = []

    try:
        open_url = urllib.request.urlopen(url)
        soup = bs(open_url, 'html.parser')
        book_links = soup.find_all('a', class_='pollAnswer__bookLink')

        # Extract URLs from book links
        for link in book_links:
            urls.append('https://goodreads.com' + link['href'])

    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

    return urls

def get_book_urls_for_list(list_of_urls):
    all_urls = []
    for url in list_of_urls:
        urls = get_book_urls(url)
        all_urls.extend(urls)
    return all_urls

In [3]:
url23='https://www.goodreads.com/choiceawards/best-books-2023'
urls2023 = get_award_urls(url23)

In [26]:
url22='https://www.goodreads.com/choiceawards/best-books-2022'
url21='https://www.goodreads.com/choiceawards/best-books-2021'
url20='https://www.goodreads.com/choiceawards/best-books-2020'
url19='https://www.goodreads.com/choiceawards/best-books-2019'

In [27]:
urls2022 = get_award_urls(url22)
urls2021 = get_award_urls(url21)
urls2020 = get_award_urls(url20)
urls2019 = get_award_urls(url19)

In [5]:
book_urls_2023 = get_book_urls_for_list(urls2023)

In [28]:
book_urls_2022 = get_book_urls_for_list(urls2022)
book_urls_2021 = get_book_urls_for_list(urls2021)
book_urls_2020 = get_book_urls_for_list(urls2020)
book_urls_2019 = get_book_urls_for_list(urls2019)

## get books info

In [6]:
def get_book_info(urls):

    books = []
    # iterate over the list of urls
    for url in urls:
        page = requests.get(url)
        soup = bs(page.content, 'html.parser')
    
        book_dict = {}
        book_dict['title'] = gr.get_title(soup)
        book_dict['ISBN'] = gr.get_ISBN(soup)
        book_dict['author'] = gr.get_author(soup)
        book_dict['genre'] = gr.get_genre(soup)
        book_dict['rating'] = gr.get_rating(soup)
        book_dict['number_of_ratings'] = gr.get_rating_count(soup)
        book_dict['number_of_reviews'] = gr.get_reviews_count(soup)
        book_dict['pages'] = gr.get_pages(soup)
        book_dict['format'] = gr.get_format(soup)
        book_dict['publish info'] = gr.get_publish_info(soup)
        book_dict['description'] = gr.get_description(soup)
    
        books.append(book_dict)
    return books

In [8]:
import pandas as pd

In [7]:
books_2023 = get_book_info(book_urls_2023)

In [9]:
books_2023_df = pd.DataFrame(books_2023)
books_2023_df.head(3)

,title,ISBN,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description
0,Yellowface,9780063250833,R.F. Kuang,"Fiction, Contemporary, Audiobook, Literary Fic...",3.80,"434,299","59,749",336,Hardcover,"First published May 25, 2023",Alternate cover edition of ISBN 9780063250833....
1,Hello Beautiful,ISBN not found,Ann Napolitano,"Fiction, Historical Fiction, Audiobook, Contem...",4.18,"300,811","28,486",416,Kindle,"First published March 14, 2023",An emotionally layered and engrossing story of...
2,The Wishing Game,ISBN not found,Meg Shaffer,"Fiction, Fantasy, Romance, Contemporary, Books...",4.12,"105,324","16,843",304,Hardcover,"First published May 30, 2023",Make a wish. . . .Lucy Hart knows better than ...
3,Tom Lake,ISBN not found,Ann Patchett,"Fiction, Audiobook, Literary Fiction, Romance,...",4.03,"245,673","30,381",309,Hardcover,"First published August 1, 2023",In this beautiful and moving novel about famil...
4,The Five-Star Weekend,ISBN not found,Elin Hilderbrand,"Fiction, Romance, Audiobook, Chick Lit, Contem...",4.08,"161,047","10,963",384,Hardcover,"First published June 13, 2023",From the #1 New York Times bestselling author ...
...,...,...,...,...,...,...,...,...,...,...,...
294,Misfit: Growing Up Awkward in the '80s,ISBN not found,Gary Gulman,"Memoir, Humor, Nonfiction, Audiobook, Comedy, ...",4.16,"2,102",304,304,Hardcover,"Published September 19, 2023",A tour de force of comedy and reflection about...
295,"Unreliable Narrator: Me, Myself, and Impostor ...",ISBN not found,Aparna Nancherla,"Memoir, Nonfiction, Humor, Essays, Audiobook, ...",3.64,"1,071",130,304,Hardcover,"First published September 19, 2023",A hilarious and insightful collection of essay...
296,America the Beautiful?: One Woman in a Borrowe...,ISBN not found,Blythe Roberson,"Nonfiction, Travel, Memoir, Humor, Nature, Aud...",3.79,"1,815",355,304,Paperback,"First published April 18, 2023","For writer and comedian Blythe Roberson, there..."
297,Alexandra Petri's US History: Important Americ...,ISBN not found,Alexandra Petri,"Humor, History, Nonfiction, Politics, Essays, ...",3.70,725,189,326,Hardcover,"First published April 11, 2023","A witty, absurdist satire of the last 500 year..."


In [33]:
books_2022 = get_book_info(book_urls_2022)

In [36]:
books_2022_df = pd.DataFrame(books_2022)

In [51]:
books_2021 = get_book_info(book_urls_2021)
books_2021_df = pd.DataFrame(books_2021)

An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found
An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found


In [53]:
books_2020 = get_book_info(book_urls_2020)
books_2020_df = pd.DataFrame(books_2020)

An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found


In [55]:
books_2019 = get_book_info(book_urls_2019)
books_2019_df = pd.DataFrame(books_2019)

An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found
An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found
An error occurred: 'NoneType' object has no attribute 'find'
Genre container not found
Rating element not found


# get number of votes

In [38]:
def get_votes(url):
    try:
        open_url = urllib.request.urlopen(url)
        soup = bs(open_url, 'html.parser')
        votes_elements = soup.find_all('strong', class_='uitext result')

        # Extract URLs from book links
        votes = []
        # Extract the number of votes from each element
        for element in votes_elements:
            if element:
                votes_text = element.text.strip()  # e.g., "200,722 votes"
                number_of_votes = votes_text.split()[0].replace(",", "")  # Remove commas and take only the number part
                votes.append(number_of_votes)
    
        return votes

    except urllib.error.HTTPError as e:
        print(f"HTTP Error: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [39]:
def get_book_votes_for_list(list_of_urls):
    all_urls = []
    for url in list_of_urls:
        urls = get_votes(url)
        all_urls.extend(urls)
    return all_urls

In [19]:
vote_list23 = get_book_votes_for_list(urls2023)

In [40]:
vote_list22 = get_book_votes_for_list(urls2022)
vote_list21 = get_book_votes_for_list(urls2021)
vote_list20 = get_book_votes_for_list(urls2020)
vote_list19 = get_book_votes_for_list(urls2019)

In [23]:
# Add the List as a New Column to dataframe
books_2023_df['Votes'] = vote_list23

books_2023_df.head(3)

,title,ISBN,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes
0,Yellowface,9780063250833,R.F. Kuang,"Fiction, Contemporary, Audiobook, Literary Fic...",3.80,"434,299","59,749",336,Hardcover,"First published May 25, 2023",Alternate cover edition of ISBN 9780063250833....,200722
1,Hello Beautiful,ISBN not found,Ann Napolitano,"Fiction, Historical Fiction, Audiobook, Contem...",4.18,"300,811","28,486",416,Kindle,"First published March 14, 2023",An emotionally layered and engrossing story of...,60171
2,The Wishing Game,ISBN not found,Meg Shaffer,"Fiction, Fantasy, Romance, Contemporary, Books...",4.12,"105,324","16,843",304,Hardcover,"First published May 30, 2023",Make a wish. . . .Lucy Hart knows better than ...,57702


In [25]:
# export csv files
books_2023_df.to_csv('../data/raw_data/books_2023_df.csv', index=False)

In [42]:
books_2022_df['Votes'] = vote_list22
books_2022_df.to_csv('../data/raw_data/books_2022_df.csv', index=False)

In [52]:
books_2021_df['Votes'] = vote_list21
books_2021_df.to_csv('../data/raw_data/books_2021_df.csv', index=False)
books_2021_df.head(3)

,title,ISBN,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes
0,"Beautiful World, Where Are You",ISBN not found,Sally Rooney,"[Fiction, Contemporary, Romance, Literary Fict...",3.54,389308,46048,356,Hardcover,"First published September 7, 2021","Alice, a novelist, meets Felix, who works in a...",69770
1,Dial A for Aunties,ISBN not found,Jesse Q. Sutanto,"[Romance, Fiction, Mystery, Contemporary, Audi...",3.72,92923,16805,299,Hardcover,"First published April 27, 2021",What happens when you mix 1 (accidental) murde...,43054
2,Cloud Cuckoo Land,ISBN not found,Anthony Doerr,"[Fiction, Historical Fiction, Fantasy, Science...",4.26,214661,28964,626,Hardcover,"First published September 28, 2021","When everything is lost, it’s our stories that...",37952


In [54]:
books_2020_df['Votes'] = vote_list20
books_2020_df.to_csv('../data/raw_data/books_2020_df.csv', index=False)
books_2020_df.head(3)

,title,ISBN,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes
0,The Midnight Library,ISBN not found,Matt Haig,"[Fiction, Fantasy, Contemporary, Audiobook, Ma...",4.00,1772068,183507,288,Hardcover,"First published August 13, 2020","Between life and death there is a library, and...",72828
1,Anxious People,ISBN not found,Fredrik Backman,"[Fiction, Contemporary, Audiobook, Mystery, Hu...",4.18,634916,77744,336,Hardcover,"First published April 25, 2019","A poignant, charming novel about a crime that ...",72823
2,American Dirt,ISBN not found,Jeanine Cummins,"[Fiction, Audiobook, Contemporary, Historical ...",4.37,572483,44868,380,Hardcover,"First published January 21, 2020","Jeanine Cummins's American Dirt, the #1 New Yo...",67226


In [56]:
books_2019_df['Votes'] = vote_list19
books_2019_df.to_csv('../data/raw_data/books_2019_df.csv', index=False)
books_2019_df.head(3)

,title,ISBN,author,genre,rating,number_of_ratings,number_of_reviews,pages,format,publish info,description,Votes
0,The Testaments,here.,Margaret Atwood,"[Fiction, Dystopia, Science Fiction, Feminism,...",4.19,352440,31334,422,Hardcover,"First published September 10, 2019",When the van door slammed on Offred's future a...,98291
1,Normal People,ISBN not found,Sally Rooney,"[Fiction, Romance, Contemporary, Literary Fict...",3.81,1388906,135364,273,Hardcover,"First published August 28, 2018",At school Connell and Marianne pretend not to ...,40081
2,Where the Forest Meets the Stars,ISBN not found,Glendy Vanderah,"[Fiction, Contemporary, Mystery, Romance, Fant...",4.14,192265,16504,332,Hardcover,"First published March 1, 2019","In this gorgeously stunning debut, a mysteriou...",29342
